In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
final_df = pd.read_csv("updated_air_quality_dataset.csv")

# Convert the combined Date to a datetime column and create day_of_week
final_df['datetime'] = pd.to_datetime(final_df['Date'])
final_df['day_of_week'] = final_df['datetime'].dt.dayofweek

# Drop the original Date and datetime columns
final_df = final_df.drop(columns=['Date', 'datetime'])

# -------------------------------------------
# Feature Engineering: Rolling Average
# -------------------------------------------
# Sort by county and time components to ensure proper ordering for rolling computation
final_df = final_df.sort_values(['County Name', 'Year', 'Month', 'Day', 'Hour'])
# Compute a rolling average for temperature over a window of 3 (adjust window as needed)
final_df['temp_roll_avg'] = final_df.groupby('County Name')['temperature_2m (°C)'] \
                                    .rolling(window=3, min_periods=1).mean() \
                                    .reset_index(level=0, drop=True)

# -------------------------------------------
# Feature Engineering: Cyclic Encoding for 'Day'
# -------------------------------------------
# Convert the 'Day' column into cyclic features (assuming a maximum of 31 days)
final_df['Day_sin'] = np.sin(2 * np.pi * final_df['Day'] / 31)
final_df['Day_cos'] = np.cos(2 * np.pi * final_df['Day'] / 31)
# Drop the original 'Day' column as its information is now captured in the cyclic features
final_df = final_df.drop(columns=['Day'])

# Update the features list to include the new cyclic encoding and rolling average columns,
# and remove the dropped 'Day' column.
features = ['Year', 'Month', 'Hour', 'day_of_week', 'Day_sin', 'Day_cos',
            'temperature_2m (°C)', 'relative_humidity_2m (%)', 
            'precipitation (mm)', 'wind_speed_100m (km/h)', 'temp_roll_avg']

target_pm25 = 'PM2.5'
target_pm10 = 'PM10'
target_no2 = 'NO2'

# Function to perform hyperparameter tuning, training, and evaluation of a RandomForestRegressor
def train_and_evaluate_rf(X_train, X_test, y_train, y_test, target_name):
    # Define the parameter grid for hyperparameter tuning
    param_grid = {
         'n_estimators': [50, 100, 200],
         'max_depth': [None, 10, 20, 30],
         'min_samples_split': [2, 5, 10],
         'min_samples_leaf': [1, 2, 4],
         'bootstrap': [True, False]
    }
    
    # Initialize the RandomForestRegressor
    rf = RandomForestRegressor(random_state=42)
    
    # Use GridSearchCV for hyperparameter tuning with 3-fold cross-validation
    grid_search = GridSearchCV(estimator=rf,
                               param_grid=param_grid,
                               cv=3,
                               n_jobs=-1,
                               scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    
    # Retrieve the best estimator
    best_rf = grid_search.best_estimator_
    
    # Predict on the test set and calculate performance metrics
    y_pred = best_rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"{target_name} Model - MSE: {mse:.3f}, R2: {r2:.3f}")
    return best_rf

###################################################################################################
# New York (Using PM2.5 and Average Traffic Density)
ny_df = final_df[final_df['County Name'] == 'New York']
# Include traffic density for New York along with our features
ny_features = features + ['Average Traffic Density']
X_ny = ny_df[ny_features]
y_ny_pm25 = ny_df[target_pm25]

print("--------------------------------------------------")
# Split the New York dataset into training and testing sets
X_ny_train, X_ny_test, y_ny_train, y_ny_test = train_test_split(X_ny, y_ny_pm25, test_size=0.2, random_state=42)
# Train and evaluate the PM2.5 model for New York with hyperparameter tuning
rf_pm25_ny = train_and_evaluate_rf(X_ny_train, X_ny_test, y_ny_train, y_ny_test, "New York PM2.5")

###################################################################################################
# Chicago (Using PM10, NO2, and PM2.5)
chicago_df = final_df[final_df['County Name'] == 'Cook']
X_chicago = chicago_df[features]
y_chicago_pm10 = chicago_df[target_pm10]
y_chicago_no2 = chicago_df[target_no2]
y_chicago_pm25 = chicago_df[target_pm25]

# Split the Chicago dataset into training and testing sets (for three target variables)
X_chicago_train, X_chicago_test, \
y_chicago_pm10_train, y_chicago_pm10_test, \
y_chicago_no2_train, y_chicago_no2_test, \
y_chicago_pm25_train, y_chicago_pm25_test = train_test_split(
    X_chicago, y_chicago_pm10, y_chicago_no2, y_chicago_pm25, test_size=0.2, random_state=42)

print("--------------------------------------------------")
# Train and evaluate the PM10 model for Chicago
rf_pm10_chicago = train_and_evaluate_rf(X_chicago_train, X_chicago_test, y_chicago_pm10_train, y_chicago_pm10_test, "Chicago PM10")
# Train and evaluate the NO2 model for Chicago
rf_no2_chicago = train_and_evaluate_rf(X_chicago_train, X_chicago_test, y_chicago_no2_train, y_chicago_no2_test, "Chicago NO2")
# Train and evaluate the PM2.5 model for Chicago
rf_pm25_chicago = train_and_evaluate_rf(X_chicago_train, X_chicago_test, y_chicago_pm25_train, y_chicago_pm25_test, "Chicago PM2.5")

###################################################################################################
# Los Angeles (Using PM10, NO2, and PM2.5)
los_angeles_df = final_df[final_df['County Name'] == 'Los Angeles']
X_la = los_angeles_df[features]
y_la_pm10 = los_angeles_df[target_pm10]
y_la_no2 = los_angeles_df[target_no2]
y_la_pm25 = los_angeles_df[target_pm25]

# Split the Los Angeles dataset into training and testing sets (for three target variables)
X_la_train, X_la_test, \
y_la_pm10_train, y_la_pm10_test, \
y_la_no2_train, y_la_no2_test, \
y_la_pm25_train, y_la_pm25_test = train_test_split(
    X_la, y_la_pm10, y_la_no2, y_la_pm25, test_size=0.2, random_state=42)

print("--------------------------------------------------")
# Train and evaluate the PM10 model for Los Angeles
rf_pm10_la = train_and_evaluate_rf(X_la_train, X_la_test, y_la_pm10_train, y_la_pm10_test, "Los Angeles PM10")
# Train and evaluate the NO2 model for Los Angeles
rf_no2_la = train_and_evaluate_rf(X_la_train, X_la_test, y_la_no2_train, y_la_no2_test, "Los Angeles NO2")
# Train and evaluate the PM2.5 model for Los Angeles
rf_pm25_la = train_and_evaluate_rf(X_la_train, X_la_test, y_la_pm25_train, y_la_pm25_test, "Los Angeles PM2.5")


--------------------------------------------------
New York PM2.5 Model - MSE: 14.173, R2: 0.563
--------------------------------------------------
Chicago PM10 Model - MSE: 50.844, R2: 0.770
Chicago NO2 Model - MSE: 13.118, R2: 0.787
Chicago PM2.5 Model - MSE: 12.501, R2: 0.622
--------------------------------------------------
Los Angeles PM10 Model - MSE: 116.538, R2: 0.357
Los Angeles NO2 Model - MSE: 24.609, R2: 0.813
